In [1]:
import sqlite3
import numpy as np
import comet_ml
# load the database path
from config import settings
from pathlib import Path

settings_proc ='SETTINGS1'
settings_simu = 'SETTINGS1'
root= Path(settings.data.path["processed"])
database_path = (root /settings_simu/settings_simu.lower()).with_suffix('.db')

In [2]:
database_path

PosixPath('/home/yacine/Documents/PhD/Code/GitProject/PBSHM_mdof/data/processed/SETTINGS1/settings1.db')

In [3]:
from psm.utils.data.metadata import get_metadata_processed
metadata= get_metadata_processed(settings_proc, settings_simu)
freq_axis = metadata['freq']

In [4]:
from psm.models.prepare_data import CreateTransformer,PSDDataModule,PSDDataset
# let's create the transformer
transformer = CreateTransformer(database_path, freq=freq_axis, freq_min=0, freq_max=150)
transform_psd = transformer.transform_psd
transform_label = transformer.transform_label
input_dim = transformer.dimension_psd()
dm = PSDDataModule(database_path, transform_psd, transform_label, batch_size=32)
dm.setup()

# Create dataloaders
train_dl = dm.train_dataloader()
val_dl = dm.val_dataloader()
test_dl = dm.test_dataloader()

# Print some details
print(f"Number of training samples: {len(dm.train_dataset)}")
print(f"Number of validation samples: {len(dm.val_dataset)}")
print(f"Number of test samples: {len(dm.test_dataset)}")

# Get a batch of data
for batch in train_dl:
    data, label = batch
    print(f"Shape of data from train_dl: {data.shape}")
    print(f"Shape of label from train_dl: {label.shape}")
    break

(anomaly_level=?) AND stage=? (0, 'train')
(anomaly_level=?) AND stage=? (0, 'test')
Number of training samples: 16000
Number of validation samples: 4000
Number of test samples: 4000
Shape of data from train_dl: torch.Size([32, 385])
Shape of label from train_dl: torch.Size([32])


In [5]:
from psm.models.prepare_data import PSDNotchDataset , PSDNotchDatasetOriginal
database_notch_path = (root/settings_simu/(settings_proc.lower()+"_vas")).with_suffix('.db')

anomaly_dl = PSDDataset(database_path, transform=transform_psd, transform_label=transform_label, stage='anomaly')
test_dl = PSDDataset(database_path, transform=transform_psd, transform_label=transform_label, stage='test')


psd_notch = PSDNotchDataset(database_notch_path, transform=transform_psd, transform_label=transform_label)
psd_original = PSDNotchDatasetOriginal(database_notch_path,transform=transform_psd, transform_label=transform_label)


stage=? ('anomaly',)
stage=? ('test',)
system_name LIKE ? ('%',)
system_name LIKE ? ('%',)


In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from psm.models.callbacks_logger import log_metrics 
from psm.models.energy_classification import EnergyDenseSignalClassifierModule 
from psm.models.callbacks_logger import create_callbacks_loggers
from pytorch_lightning import Trainer
from psm.models.ad_systems import AD_energy
from psm.eval.benchmark import Benchmark_SA, Benchmark_VAS
from torch import nn

# create logger and callbacks
callbacks, logger = create_callbacks_loggers()
# hyperparameters
hyper_params = {'input_dim':input_dim,'en_coef':0.3, 'dense_layers':[512,248,248,128, 64, 32],
                'dropout_rate':0, 'num_classes':20, 'lr':0.002,
                'batch_norm':True, 'activation':nn.ReLU(), 'l1_reg':1e-4}
# create model
model = EnergyDenseSignalClassifierModule(**hyper_params)

trainer = Trainer(max_epochs=100, callbacks=callbacks, logger=logger)
trainer.fit(model, dm)

checkpoint_path = trainer.checkpoint_callback.best_model_path
best_model = EnergyDenseSignalClassifierModule.load_from_checkpoint(checkpoint_path)

trainer.test(best_model, dm)
ad_system = AD_energy(model=best_model.model)
ad_system.fit(train_dl)
#ad_system.log_model(logger.experiment)
logger.experiment.log_model("best_model", checkpoint_path)
logger.experiment.log_asset(checkpoint_path,step=trainer.global_step)


# first benchmark
benchmark1= Benchmark_SA(ad_system,anomaly_dl, test_dl, batch_size=10000)
res1 = benchmark1.evaluate()
benchmark2= Benchmark_VAS(ad_system,psd_notch,psd_original,batch_size=10000)
res2 = benchmark2.evaluate_all_systems()
global_metric = log_metrics(logger,res1,res2)

# After training, you can also test your model

logger.experiment.end()

CometLogger will be initialized in online mode
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
